In [1]:
import pandas as pd
from geotext import GeoText
import numpy as np

In [2]:
df = pd.read_csv('omni.csv')

/Users/murielhol/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Source 1:  the desciption in the dataset itself

In [3]:
cities_description, countries_description = [], []
from collections import Counter

In [4]:
for i, row in df.iterrows():
    text = row.description
    if text and not str(text)=='nan':
        text = text.title().replace(' ', ', ')
        gt = GeoText(text)
        try:
            cit = Counter([ c for c in gt.cities if not c in ['Of', 'Man', 'Same', 'Much', 'Date', 'Most', 'Van']]).most_common(1)[0][0]
            cou = Counter([r for r in gt.country_mentions if not r == 'TR' ]).most_common(1)[0][0]
            cities_description.append(cit)
            countries_description.append(cou)
        except:
            cities_description.append('')
            countries_description.append('')
    else:
        cities_description.append('')
        countries_description.append('')


In [5]:
print(cities_description[:100])

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'London', '', '', 'Chicago', '', '', '', '', '', '', '', '', '', 'Roman', '', '', 'Young', 'Alexandria', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Hemet', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


## source 2: wikipedia crawls

In [6]:
df_ = df.copy().drop_duplicates(subset='artist_full_name')
full_names = list(df_.artist_full_name)
first_names = list(df_.artist_first_name)
last_names = list(df_.artist_last_name)

In [7]:
print(len(df), len(df_))

241634 57297


In [8]:
e = 0
cities_wiki, countries_wiki, wiki_match_artists = [], [], []
wiki = pd.read_csv('wikipedia_locations.csv')
for i in range(len(last_names)):
    check = False
    try:
        full_name = full_names[i]
        name = last_names[i]
        name = str(name).replace('(',"").replace(')', "").replace('[',"").replace(']', "").split()
        first_name = first_names[i]
        first_name = str(first_name).replace('(',"").replace(')', "").replace('[',"").replace(']', "").split()
        # reduce the search in 3 steps

        if len(name[-1])>1:
            this_df = wiki[wiki['artist'].str.contains(name[-1], regex=False, case=False)]
            check=True
    #             if len(this_df)>0:
    #                 print(name[-1], full_name)
    #                 print(this_df)
    #                 print('\n \n \n')


        if len(name[0])>1:
            check=True
            this_df = this_df[this_df['artist'].str.contains(name[-0], case=False, regex=False)]


        if len(first_name[0])>1:
            check=True
            this_df = this_df[this_df['artist'].str.contains(first_name[0], case=False, regex=False)]

        if check and len(this_df)>1 and len(this_df)<10:
            options = [a.replace('-', ' ') for a in this_df.artist]
            scores = [int(a.strip()==full_name.strip()) for a in options]
            if 1 in scores:
                best = np.amax(scores)
                this_df = this_df.iloc[best]

        if len(this_df)==1:
            wiki_match_artists.append(full_names[i])
            cities_wiki.append(list(this_df.city.values))
            countries_wiki.append(list(this_df.country.values))    
    


    except:
        e+=1
        pass
    
print('fails:', e)

fails: 0


In [9]:
df_wiki = pd.DataFrame({'artist': wiki_match_artists, 'country':countries_wiki, 'city':cities_wiki})

In [10]:
print(df_wiki.head(20))
print(len(df_wiki))

                    artist            city country
0          wyndham lewis         [Amherst]    [CA]
1           rudolf bauer             [nan]    [PL]
2   james mcneill whistler        [Lowell]    [US]
3        morton shamberg    [Philadelphia]    [US]
4              paul klee             [nan]    [CH]
5       kazimir malevich            [Kiev]    [RU]
6      wassily kandinsky          [Moscow]    [US]
7        robert delaunay           [Paris]    [US]
8       carl buchheister         [Hanover]    [DE]
9        kurt schwitters         [Hanover]    [US]
10          andre masson           [Lille]    [FR]
11    sophie taeuber-arp             [nan]    [CH]
12           anni albers          [Berlin]    [DE]
13            vija celmins          [Riga]    [LV]
14         gerhard richter       [Dresden]    [DE]
15            lucian freud        [Berlin]    [DE]
16            stuart davis  [Philadelphia]    [US]
17        marcel duchamp             [nan]    [FR]
18       francis picabia       

## source 3: The RDF dbpedia queries

In [11]:
pedia = pd.read_csv('dbpedia_locations.csv').drop_duplicates(subset=['artist'])

In [12]:
cities_pedia, countries_pedia, pedia_match_artists = [], [], []
for i in range(len(last_names)):
    check = False
    try:
        full_name = full_names[i]
        name = last_names[i]
        name = str(name).replace('(',"").replace(')', "").replace('[',"").replace(']', "").split()
        first_name = first_names[i]
        first_name = str(first_name).replace('(',"").replace(')', "").replace('[',"").replace(']', "").split()
        # reduce the search in 3 steps
        
        if len(name[-1])>1:
            this_df = pedia[pedia['artist'].str.contains(name[-1], regex=False, case=False)]
            check=True
#             if len(this_df)>0:
#                 print(name[-1], full_name)
#                 print(this_df)
#                 print('\n \n \n')
                
        
        if len(name[0])>1:
            check=True
            this_df = this_df[this_df['artist'].str.contains(name[-0], case=False, regex=False)]

        
        if len(first_name[0])>1:
            check=True
            this_df = this_df[this_df['artist'].str.contains(first_name[0], case=False, regex=False)]


        if len(this_df)==1:
            pedia_match_artists.append(full_names[i])
            cities_pedia.append(list(this_df.city.values))
            countries_pedia.append(list(this_df.country.values))    

    except:
        e+=1
        pass
print('fails:', e)

fails: 0


In [13]:
df_pedia = pd.DataFrame({'artist': pedia_match_artists, 'country':countries_pedia, 'city':cities_pedia})

In [14]:
print(df_pedia.head())

                artist          city   country
0  wassily kandinsky        [Moscow]      [US]
1         franz marc        [Munich]      [DE]
2        pablo picasso      [M?laga]  [M?laga]
3       martin puryear  [Washington]      [US]
4          max kozloff     [Chicago]      [US]


## Combine : 
- First see if the wiki and dbpedia match in country
    - if so: accept country
    - if not: check if they match city

In [15]:
full_names_all = list(df.artist_full_name)

In [16]:
countries, cities, artists = [], [], []

cw, cwp, cpd, cwd, cp, cd = 0,0,0,0,0,0

for i in range(len(full_names_all)):
    fn = full_names_all[i]
    wiki = df_wiki[df_wiki.artist==fn]
    pedia = df_pedia[df_pedia.artist==fn]
    
    # countries
    if len(wiki)>0 and len(pedia)>0:
        if str(wiki.country.values[0][0]) == str(pedia.country.values[0][0]):
            countries.append(wiki.country.values[0][0])
            cwp+=1
        elif str(wiki.country.values[0][0]) in countries_description[i]:
            countries.append(wiki.country.values[0][0])
            cwd+=1
        elif str(pedia.country.values[0][0]) in countries_description[i]:
            countries.append(pedia.country.values[0][0])
            cpd+=1
        else:
            countries.append(wiki.country.values[0][0])
            cw+=1
    elif len(wiki)>0:
        cw+=1
        countries.append(wiki.country.values[0][0])
    elif len(pedia)>0:
        cp+=1
        countries.append(pedia.country.values[0][0])
    elif len(countries_description[i])>0:
        cd+=1
        countries.append(countries_description[i])
    else:
        countries.append('')
        
        
    # cities
    if len(wiki)>0 and len(pedia)>0:
        if str(wiki.city.values[0][0]) == str(pedia.city.values[0][0]):
            cities.append(wiki.city.values[0][0])
            cwp+=1
        elif str(wiki.city.values[0][0]) in str(cities_description[i]):
            cities.append(wiki.city.values[0][0])
            cwd+=1
        elif str(pedia.city.values[0][0]) in cities_description[i]:
            cities.append(pedia.city.values[0][0])
            cpd+=1
        else:
            cities.append(wiki.city.values[0][0])
            cw+=1
    elif len(wiki)>0:
        cw+=1
        cities.append(wiki.city.values[0][0])
    elif len(pedia)>0:
        cp+=1
        cities.append(pedia.city.values[0][0])
    elif len(cities_description[i])>0:
        cd+=1
        cities.append(cities_description[i])
    else:
        cities.append('')
        
print('cw, cwp, cpd, cwd, cp, cd')
print(cw, cwp, cpd, cwd, cp, cd)
    

cw, cwp, cpd, cwd, cp, cd
39762 2027 5 0 6004 534


In [31]:
df['country'] = countries

In [18]:
df['city'] = cities

In [44]:
df.city[100:120]

100    
101    
102    
103    
104    
105    
106    
107    
108    
109    
110    
111    
112    
113    
114    
115    
116    
117    
118    
119    
Name: city, dtype: object

In [51]:
df_nc = df[df.city!='']
df_nc = df_nc[df_nc.country!='']

In [52]:
print(len(df), len(df_nc))

241634 24166


In [54]:
print(df_nc.country[:10])

15    CA
16    CA
18    PL
19    US
20    US
22    US
24    US
32    RO
35    UY
36    IT
Name: country, dtype: object


In [60]:
import time
cities = df_nc.city
countries = df_nc.country
long, lat = [], []
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
N = len(countries)
for i in range(N):
    try:
        location = geolocator.geocode(cities[i]+' , '+countries[i])
        lat.append(location.latitude)
        long.append(location.longitude)
    except:
        lat.append('')
        long.append('')
        
    if i%100==0:
        print(i/N)
        time.sleep(100)
        


0.0
0.004138045187453447
0.008276090374906894
0.01241413556236034
0.016552180749813787
0.020690225937267236
0.02482827112472068
0.02896631631217413
0.033104361499627574
0.03724240668708102
0.04138045187453447
0.04551849706198792
0.04965654224944136
0.05379458743689481
0.05793263262434826
0.06207067781180171
0.06620872299925515
0.0703467681867086
0.07448481337416205
0.0786228585616155
0.08276090374906894
0.08689894893652239
0.09103699412397584
0.09517503931142927
0.09931308449888272
0.10345112968633617
0.10758917487378962
0.11172722006124307
0.11586526524869652
0.12000331043614997
0.12414135562360341
0.12827940081105685
0.1324174459985103
0.13655549118596375
0.1406935363734172
0.14483158156087064
0.1489696267483241
0.15310767193577754
0.157245717123231
0.16138376231068444
0.16552180749813789
0.16965985268559133
0.17379789787304478
0.17793594306049823
0.18207398824795168
0.18621203343540513
0.19035007862285855
0.194488123810312
0.19862616899776545
0.2027642141852189
0.20690225937267234
0

In [63]:
df_nc['long'] = long
df_nc['lat'] = lat

In [ ]:
df_nc

In [64]:
print(len(df_nc[df_nc.lat!='']))

10425


In [65]:
print(len(df_nc))

24166


In [66]:
last_df = df_nc[df_nc.lat!='']

In [85]:
final = last_df.copy()

In [86]:
eg = geolocator.geocode('Egypt, Caïro ')
ro = geolocator.geocode('Italy, Rome')
gr = geolocator.geocode('Greek, Athene')

In [92]:
final.loc[final.artist_last_name=='roman', 'country'] =  'IT'
final.loc[final.artist_last_name=='roman', 'city'] =  'Rome'
final.loc[final.artist_last_name=='roman', 'long'] =  ro.longitude
final.loc[final.artist_last_name=='roman', 'lat'] =  ro.latitude


final.loc[final.artist_last_name=='greek', 'country'] =  'GR'
final.loc[final.artist_last_name=='greek', 'city'] =  'Athene'
final.loc[final.artist_last_name=='greek', 'long'] =  gr.longitude
final.loc[final.artist_last_name=='greek', 'lat'] =  gr.latitude


final.loc[final.artist_last_name=='egyptian', 'country'] =  'EG'
final.loc[final.artist_last_name=='egyptian', 'city'] =  'Caïro'
final.loc[final.artist_last_name=='egyptian', 'long'] =  eg.longitude
final.loc[final.artist_last_name=='egyptian', 'lat'] =  eg.latitude

In [93]:
print(final.tail(100)[['artist_last_name',  'country']])

         artist_last_name                     country
241205     adrian-nilsson                          SE
241207         baumeister                          DE
241210         baumeister                          DE
241214           van gogh                          NL
241215         servranckx                          BE
241220              roman                          IT
241221              roman                          IT
241222              roman                          IT
241223       lloyd wright  Richland Center, Wisconsin
241226              roman                          IT
241227       lloyd wright  Richland Center, Wisconsin
241228              weese                          US
241230       lloyd wright  Richland Center, Wisconsin
241231       lloyd wright  Richland Center, Wisconsin
241234       lloyd wright  Richland Center, Wisconsin
241235       lloyd wright  Richland Center, Wisconsin
241240       lloyd wright  Richland Center, Wisconsin
241241       lloyd wright  R

In [94]:
final.to_csv('omni_locations.csv')

In [95]:
cols = final.columns

In [98]:
for col in cols:
    print(col, len(set(final[col])))

Unnamed: 0 10425
omni_id 10425
artwork_name 7091
artist_full_name 1327
artist_first_name 906
artist_last_name 1242
date 536
century 15
source_url 7279
image_url 8079
general_type 72
genre 109
school 55
palette_families 337
palette 8074
geocoded 1
media 252
style 252
location 380
tag 1507
dominant_color 7487
dominant_color_family 10
description 902
county 162
city 639
country 161
long 480
lat 480


In [99]:
for col in cols:
    print(col, final[col][:10])
    print('\n')

Unnamed: 0 130    1125
135    1138
141    1157
143    1165
148    1181
151    1334
159    1569
252    1751
256    1800
283    2145
Name: Unnamed: 0, dtype: int64


omni_id 130       3935
135    9006173
141        356
143    9006198
148    9006214
151       1007
159    9006597
252    9006775
256    9006824
283    9007167
Name: omni_id, dtype: int64


artwork_name 130                                                  304
135                                amphora (storage jar)
141                                    homage to bleriot
143                             kantharos (drinking cup)
148                           knob-handled patera (dish)
151                         composition tight curves. 25
159                     violet and silver - the deep sea
252    pair of protomes depicting the forepart of a g...
256                           trefoil oinochoe (pitcher)
283                              sunny morningeight legs
Name: artwork_name, dtype: object


artist_full_name 130       wa

In [105]:
print(final[2500:2600][['artist_full_name', 'city', 'country']])

               artist_full_name        city country
12177              salvador dal    Figueres      ES
12178       annibale carracci       Bologna      IT
12179               paul bril         Breda      NL
12180               paul bril         Breda      NL
12181               paul bril         Breda      NL
12182               paul bril         Breda      NL
12186               paul bril         Breda      NL
12187               paul bril         Breda      NL
12192       hendrick avercamp     Amsterdam      US
12195               paul bril         Breda      NL
12196       hendrick avercamp     Amsterdam      US
12197     esaias van de velde     Amsterdam      US
12205       hendrick avercamp     Amsterdam      US
12208       hendrick avercamp     Amsterdam      US
12209       hendrick avercamp     Amsterdam      US
12217         jan dirksz both       Utrecht      NL
12219                lan ying      Hangzhou      CN
12221      jacob van ruisdael       Haarlem      NL
12222       